# Computational Physics - Term Project

# Monte Carlo Decay Model of Trans-Uranic Elements

This project will use monte carlo decays to simulate the decay chain of a given trans-uranic element. The data will come from Brookhaven National Laboratory's NuDat 3.0 database. Data for all trans-uranic elements using NuDat 3.0 should be downloaded and supplied with this code for it to run properly.

### Importing necessary libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Data pre-processing

While NuDat 3.0 contains hundreds of rows of data on all of the trans-uranic elements, some of these rows are incomplete are useless to us when it comes to code. The data must not only be read by the code, but also pre-processed, trimming out the unnecessary or incomplete information so that our decay chain does not run into errores down the line. This will inevitably cut down on the amount of decay chains we can access or even complete, but otherwise the fully realized decay chain must be fed into the program at the start (which defeats the purpose of this program; being able to calculate the entire decay chain using solely one set of input parameters). Due to this, the results of this program will either end in the decay products leaving the unstable region of the periodic table, or settle into a psuedo-stable/unmapped region of the trans-uranic elements.

In [4]:
fileline = os.getcwd()
loc = fileline + '\\NuData.csv'
data = pd.read_csv(loc)
data

,z,n,name,levelEnergy(MeV),halflife,halflifeUnit,halflifeUncertainty,decayModes,abundance,abundanceUncertainty,betaMinus(keV),betaMinusUncertainty,electronCapture(keV),electronCaptureUncertainty,positronEmission(keV),positronEmissionUncertainty,alpha(keV),alphaUncertainty
0,92,123,215U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7080.0,13,6060.0,13,8590.0,6.0
1,92,130,222U,0.0,4.70,us,2,A AP 100.00,NaN,NaN,-7000.0,6,2210.0,10,1190.0,10,9480.0,5.0
2,92,132,224U,0.0,0.84,ms,18,A = 100.00,NaN,NaN,-6290.0,3,1880.0,17,858.0,17,8628.0,7.0
3,92,131,223U,0.0,18.00,us,5,A = 100.00,NaN,NaN,-4610.0,10,3710.0,10,2690.0,10,9158.0,17.0
4,92,131,223U,0.0,18.00,us,5,EC = 0.20,NaN,NaN,-4610.0,10,3710.0,10,2690.0,10,9158.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,117,174,291Ts,NaN,NaN,NaN,NaN,A ?,NaN,NaN,NaN,NaN,4400.0,9,3400.0,9,11500.0,4.0
839,117,177,294Ts,0.0,0.08,s,33,A = 100.00,NaN,NaN,-2900.0,8,NaN,NaN,NaN,NaN,11200.0,4.0
840,118,175,293Og,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4400.0,11,3400.0,11,11900.0,5.0
841,118,176,294Og,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2900.0,8,1900.0,8,11900.0,3.0


,z,n,name,levelEnergy(MeV),halflife,halflifeUnit,halflifeUncertainty,decayModes,abundance,abundanceUncertainty,betaMinus(keV),betaMinusUncertainty,electronCapture(keV),electronCaptureUncertainty,positronEmission(keV),positronEmissionUncertainty,alpha(keV),alphaUncertainty
588,105,152,257Db,0.0,2.30,s,2,A AP 94.00,NaN,NaN,NaN,NaN,4290.0,16,3270.0,16,9206.0,20.0
589,105,152,257Db,0.0,2.30,s,2,SF LE 6.00,NaN,NaN,NaN,NaN,4290.0,16,3270.0,16,9206.0,20.0
590,105,152,257Db,0.0,0.67,s,6,A AP 87.00,NaN,NaN,NaN,NaN,4290.0,16,3270.0,16,9206.0,20.0
591,105,152,257Db,0.0,0.67,s,6,SF LE 13.00,NaN,NaN,NaN,NaN,4290.0,16,3270.0,16,9206.0,20.0


In [4]:
tl = []
for val in data['decayModes']: # testing to see the completeness of the data set and keeping the rows that have the necessary
                               # information
    if '=' in str(val):
        tl.append(True)
    elif 'AP' in str(val):
        val.replace('AP','=')
        tl.append(True)
    elif 'LE' i
    else:
        tl.append(False)
        
newdat = data[tl]
kdecays = newdat[newdat['halflife']>0] # keeping rows with actual half life values
kdecays = kdecays.reset_index(drop=True)
indx = kdecays['halflifeUnit']
units = {'ms' : 1e-3, 'us' : 1e-6, 'y' : 3.16e7,  'm' : 60, 'd' : 86400, 'ns' : 1e-9, 's' : 1, 'h' : 3600} # changing units
newhlu = pd.Series([units[val] for val in indx])
kdecays['halflifeUnit'] = newhlu # updating half life values
DC = [0.693/val for val in kdecays['halflife']]
kdecays['halflifeUncertainty'] = [DC[i]/newhlu[i] for i in range(len(DC))]
kdecays = kdecays.rename(columns={'halflifeUncertainty':'Decay Constants'}) # creating a column for decay constants
kdecays

,z,n,name,levelEnergy(MeV),halflife,halflifeUnit,Decay Constants,decayModes,abundance,abundanceUncertainty,betaMinus(keV),betaMinusUncertainty,electronCapture(keV),electronCaptureUncertainty,positronEmission(keV),positronEmissionUncertainty,alpha(keV),alphaUncertainty
0,92,132,224U,0.0,0.84,0.001000,825.000000,A = 100.00,NaN,NaN,-6290.0,3,1880.0,17,858.0,17,8628.0,7.0
1,92,131,223U,0.0,18.00,0.000001,38500.000000,A = 100.00,NaN,NaN,-4610.0,10,3710.0,10,2690.0,10,9158.0,17.0
2,92,131,223U,0.0,18.00,0.000001,38500.000000,EC = 0.20,NaN,NaN,-4610.0,10,3710.0,10,2690.0,10,9158.0,17.0
3,92,134,226U,0.0,268.00,0.001000,2.585821,A = 100.00,NaN,NaN,-5490.0,10,1295.0,16,273.0,16,7701.0,4.0
4,92,133,225U,0.0,95.00,0.001000,7.294737,A = 100.00,NaN,NaN,-4250.0,9,3020.0,8,1990.0,8,8007.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,116,175,291Lv,0.0,6.30,0.001000,110.000000,A = 100.00,NaN,NaN,-4400.0,9,3100.0,10,2000.0,10,10900.0,9.0
402,116,176,292Lv,0.0,18.00,0.001000,38.500000,A = 100.00,NaN,NaN,-5500.0,10,1500.0,10,0.0,AP,10791.0,12.0
403,116,177,293Lv,0.0,53.00,0.001000,13.075472,A = 100.00,NaN,NaN,-3900.0,9,NaN,NaN,NaN,NaN,10700.0,6.0
404,117,176,293Ts,0.0,14.00,0.001000,49.500000,A = 100.00,NaN,NaN,-4400.0,11,3900.0,9,2800.0,9,11300.0,5.0


We can now start working on the actual decay scheme of the nucleides. The mother nucleus is the first element in the chain, which has a different decay scheme since there won't be more of itself being generated throughout the decays. This function finds the nucleus (or nucleides) that correspond to the given input, and moves through time given a step value dt. 


For each step we generate a random number between 0 and 1, and check to see if it is greater than the normalized probability of this nucleus decaying at this time step. This probability is given by

In [7]:
def mother(data,stamp,dt,N=1): # this defines the decay chain of the element passed in, which is unique
    t = dt # decay step timer
    dbp = {'A ' : 0.0, 'B- ' : 0.0, 'EC ' : 0.0, 'IT ' : 0.0} # decay bi-products
    DT = 0 # time of decay
    RI = data[data['name']==stamp]
    RI = RI.reset_index(drop=True) # relevant isotopes
    RDC = RI['Decay Constants'][0] # relevant decay constant
    poss = len(RI)
    dm = '' # decay mode
    while DT == 0:
        if np.random.random() > np.exp(-RDC*t): # monte carlo 
            DT = t
            if poss > 1: # multi-mode decay method
                chance = np.random.random()
                prob = 0
                for iso in RI['decayModes']:
                    idn = iso.split('=')
                    prob += float(idn[1])/100
                    if prob > chance:#
                        dm = idn[0]
                        break # breaking the chain after the first method passes
            else: # if only one decay mode, then move forward with that regardless of percentage
                idn = RI['decayModes'][0].split('=')
                dm = idn[0]
        
        t += dt

    
    
                
    E = RI['alpha(keV)'][0] # energy of that decay
            
    return N2, DT, dm, E

spop = 100 # beginning population
N2, DT, DC, E = mother(kdecays,'291Lv',0.01)

N3 = spop - N2 # population of decay bi-products

In [9]:
def decayproducts(data,dm,dt,N):
    {'A ' : [4.0,2.0], 'B- ' : [0.0,1.0], 'EC ' : 0.0, 'IT ' : 0.0}

-0.10000000000000009

In [122]:
kdecays.keys()

Index(['z', 'n', 'name', 'levelEnergy(MeV)', 'halflife', 'halflifeUnit',
       'Decay Constants', 'decayModes', 'abundance', 'abundanceUncertainty',
       'betaMinus(keV)', 'betaMinusUncertainty', 'electronCapture(keV)',
       'electronCaptureUncertainty', 'positronEmission(keV)',
       'positronEmissionUncertainty', 'alpha(keV)', 'alphaUncertainty'],
      dtype='object')